In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("MY_SPENDINGS").getOrCreate()

In [ ]:
df=spark.read.csv("statement.csv",header=True,inferSchema=True).fillna(0)

#change Date from string to_date, rename column names with special charaters
#Note: MM is capital not mm(minutes)
df=df.select(f.to_date(df.Date,'dd/MM/yy').alias("Date"),"Narration",\
             f.col("`Withdrawal Amt.`").alias("Withdrawal"),\
             f.col("`Chq./Ref.No.`").alias("Ref_Number"),\
             f.col("`Deposit Amt.`").alias("Deposit"),"Closing Balance")

In [ ]:
#Use Case : Removal of specific data
df=df.filter((df.Deposit != 30000) & (df.Withdrawal != 30000) & (df.Deposit != 144807))

    #Interesting that only &(and) and |(or) works here for removing two different ... \
    #... entires, where one have 30000 in Withdrawal and a different one have 30000 in Deposit

In [ ]:
import calendar
from pyspark.sql.types import IntegerType

#Use Case : Monthly saving vs spending

#calendar.monthcalendar takes year and month as input. [-1] to reverse month lists and [0] pick last one
last_day = f.udf(lambda x,y: max(calendar.monthcalendar(int(x),int(y))[-1:][0][:5]) )

#dfs = dfs.withColumn("Last_Workingday",last_day((f.split(f.col("Date"),"-")[0]),\
#                                                (f.split(f.col("Date"),"-")[1])))

df = df.withColumn("Last_Workingday",last_day((f.date_format(df.Date,'yyyy')),\
                                                 (f.date_format(df.Date,'MM'))))

#Aug 31,2022 was an holiday. So check for a day less than that in general
#dfs = dfs.filter(f.split(f.col("Date"),"-")[2] >= ( (f.col("Last_Workingday")) - (f.lit(1)) ))

df = df.withColumn("year_month",\
                 f.when(f.date_format(df.Date,'dd') != df.Last_Workingday,f.date_format(df.Date,"yyyy-MM")).\
                 otherwise( f.date_format(f.date_add(df.Date,5),"yyyy-MM") ))
df.show(100)

In [ ]:
#Use Case : Monthly saving vs spending

#remove unwanted columns
df1 = df.select("Date","Narration","Deposit")

#Get Salary credited date, filter SALARY
dfs = df1.filter(df1.Narration.contains("SALARY"))

#make sure the amount was deposited on the last weekday/last working day of the month. Using UDF
#calendar.monthcalendar takes year and month as input. [-1] to reverse month lists and [0] pick last one
last_day = f.udf(lambda x,y: max(calendar.monthcalendar(int(x),int(y))[-1:][0][:5]) )

#dfs = dfs.withColumn("Last_Workingday",last_day((f.split(f.col("Date"),"-")[0]),\
#                                                (f.split(f.col("Date"),"-")[1])))

dfs = dfs.withColumn("Last_Workingday",last_day((f.date_format(df.Date,'yyyy')),\
                                                 (f.date_format(df.Date,'MM'))))

#Aug 31,2022 was an holiday. So check for a day less than that in general
dfs = dfs.filter(f.split(f.col("Date"),"-")[2] >= ( (f.col("Last_Workingday")) - (f.lit(1)) ))
#dfs.show()

In [ ]:
dfs.select("*",(f.split(f.col("Date"),"-")).alias("New_column")). \
        select("*",f.col("New_column")[0].alias("year"),f.col("New_column")[1].alias("month")).\
        drop("New_Column")

dfs.select("*",last_day((f.split(f.col("Date"),"-")[0]),(f.split(f.col("Date"),"-")[1])).\
                                                                  alias("New_column")).show()

In [ ]:
dfS.withColumn("last_day",f.udf(lambda x: max(
                                    calendar.monthcalendar(dfS["Date"].year(),dfS["Date"].month())[-1:][0][:5]),
                                IntegerType())
              ).show()

In [ ]:
#dfS.filter(f.split(dfS.Date,'-').getItem(2) == \
#               last_working_day(f.split(dfS.Date,'-').getItem(0),f.split(dfS.Date,'-').getItem(1)))

In [ ]:
dfS.withColumn("last_day",f.udf(lambda x: max( 
                                              calendar.monthcalendar(x.split("-")[0], x.split("-")[1]) 
                                              [-1:][0][:5] 
                                             ) 
                               ) (df.Date)
              ).show()